In [1]:
### helper.py
import math
def batches(batch_size, features, labels):
    """
    Create batches of features and labels
    :param batch_size: The batch size
    :param features: List of features
    :param labels: List of labels
    :return: Batches of (Features, Labels)
    """
    assert len(features) == len(labels)
    outout_batches = []
    
    sample_size = len(features)
    for start_i in range(0, sample_size, batch_size):
        end_i = start_i + batch_size
        batch = [features[start_i:end_i], labels[start_i:end_i]]
        outout_batches.append(batch)
        
    return outout_batches


In [3]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np



n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)
learning_rate = 0.001

# Import MNIST data
mnist = input_data.read_data_sets('/datasets/ud730/mnist', one_hot=True)

# The features are already sclaed and the data is shuffled
train_features = mnist.train.images
valid_features = mnist.validation.images
test_features  = mnist.test.images

train_labels = mnist.train.labels.astype(np.float32)
valid_labels = mnist.validation.labels.astype(np.float32)
test_labels  = mnist.test.labels.astype(np.float32)

# Features and Labels
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

# Weights & bias
weights = tf.Variable(tf.random_normal([n_input, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))

# Logits - xW + b
logits = tf.add(tf.matmul(features, weights), bias)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Extracting /datasets/ud730/mnist/train-images-idx3-ubyte.gz
Extracting /datasets/ud730/mnist/train-labels-idx1-ubyte.gz
Extracting /datasets/ud730/mnist/t10k-images-idx3-ubyte.gz
Extracting /datasets/ud730/mnist/t10k-labels-idx1-ubyte.gz


In [4]:
def print_epoch_stats(epoch_i, sess, last_features, last_labels):
    """
    Print cost and validation accuracy of an epoch
    """
    current_cost = sess.run(
        cost,
        feed_dict={features: last_features, labels: last_labels})
    valid_accuracy = sess.run(
        accuracy,
        feed_dict={features: valid_features, labels: valid_labels})
    print('Epoch: {:<4} - Cost: {:<8.3} Valid Accuracy: {:<5.3}'.format(
        epoch_i,
        current_cost,
        valid_accuracy))

In [7]:
batch_size = 128
assert batch_size is not None, 'You must set the batch size'

epochs = 10

init = tf.global_variables_initializer()

train_batches = batches(batch_size, train_features, train_labels)

with tf.Session() as sess:
    sess.run(init)
    
    # Training cycle
    for epoch_i in range(epochs):
        
        # Train optimizer on all batches
        for batch_features, batch_labels in train_batches:
            sess.run(optimizer, feed_dict={features: batch_features, labels: batch_labels})
        
        # Print cost and validation accuracy of an epoch
        print_epoch_stats(epoch_i, sess, batch_features, batch_labels)
    
    # Calculate accuracy for test dataset
    test_accuracy = sess.run(
        accuracy,
        feed_dict={features: test_features, labels: test_labels})
    
print('Test Accuracy: {}'.format(test_accuracy))

Epoch: 0    - Cost: 13.8     Valid Accuracy: 0.104
Epoch: 1    - Cost: 12.8     Valid Accuracy: 0.114
Epoch: 2    - Cost: 12.1     Valid Accuracy: 0.125
Epoch: 3    - Cost: 11.3     Valid Accuracy: 0.141
Epoch: 4    - Cost: 10.7     Valid Accuracy: 0.153
Epoch: 5    - Cost: 10.1     Valid Accuracy: 0.164
Epoch: 6    - Cost: 9.59     Valid Accuracy: 0.179
Epoch: 7    - Cost: 9.11     Valid Accuracy: 0.196
Epoch: 8    - Cost: 8.67     Valid Accuracy: 0.209
Epoch: 9    - Cost: 8.27     Valid Accuracy: 0.222
Test Accuracy: 0.23170000314712524
